# SageMaker Principal Component Analysis (PCA) Exercise

This notebook demonstrates Amazon SageMaker's **Principal Component Analysis (PCA)** algorithm for dimensionality reduction.

## What You'll Learn
1. How to prepare data for PCA
2. How to configure and understand PCA hyperparameters
3. How to train a PCA model
4. How to interpret and use principal components

## What is PCA?

PCA is an **unsupervised** algorithm that:
- Reduces high-dimensional data to fewer dimensions
- Finds directions of maximum variance (principal components)
- Components are orthogonal and ordered by explained variance
- Preserves as much information as possible in fewer dimensions

**SageMaker's Implementation:**
- Two modes: `regular` and `randomized`
- GPU acceleration for large datasets
- Single-pass streaming algorithm
- Near-linear scalability

## Use Cases

| Application | Description |
|-------------|-------------|
| Feature reduction | Reduce features before ML model |
| Visualization | Project high-dim data to 2D/3D |
| Noise reduction | Keep only high-variance components |
| Anomaly detection | Outliers in reduced space |
| Data compression | Store/transmit fewer features |

---

## ⚠️ Training Cost Information

<div style="background-color: #020703ff; border: 1px solid #28a745; border-radius: 5px; padding: 15px; margin: 10px 0;">

### PCA Supports CPU and GPU

PCA training is computationally efficient. GPU helps with very large matrices.

| Instance Type | Type | On-Demand Price* | Best For |
|---------------|------|------------------|----------|
| ml.m5.large | CPU | ~$0.13/hour | Small-medium data |
| ml.c5.xlarge | CPU | ~$0.24/hour | Medium data |
| ml.p2.xlarge | GPU | ~$1.26/hour | Large data, many features |
| ml.p3.2xlarge | GPU | ~$3.83/hour | Very large datasets |

*Prices are approximate for us-west-2.

### Cost Estimation
- **Training**: Very fast! Usually 2-5 minutes (~$0.01-0.03)
- **Inference endpoint**: ~$0.13/hour for ml.m5.large
- PCA is one of the most cost-effective SageMaker algorithms

### Mode Selection
- Use `regular` mode for moderate features (<5000)
- Use `randomized` mode for many features (>5000) or samples

</div>

## Step 1: Setup and Imports

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

# Load environment variables from .env file
load_dotenv()

# Configure AWS session from environment variables
aws_profile = os.getenv('AWS_PROFILE')
aws_region = os.getenv('AWS_REGION', 'us-west-2')
sagemaker_role = os.getenv('SAGEMAKER_ROLE_ARN')

if aws_profile:
    boto3.setup_default_session(profile_name=aws_profile, region_name=aws_region)
else:
    boto3.setup_default_session(region_name=aws_region)

# SageMaker session and role
sagemaker_session = sagemaker.Session()

if sagemaker_role:
    role = sagemaker_role
else:
    role = get_execution_role()

region = sagemaker_session.boto_region_name

print(f"AWS Profile: {aws_profile or 'default'}")
print(f"SageMaker Role: {role}")
print(f"Region: {region}")
print(f"SageMaker SDK Version: {sagemaker.__version__}")

In [ ]:
# Configuration
BUCKET_NAME = sagemaker_session.default_bucket()
PREFIX = "pca"

# Dataset parameters
NUM_SAMPLES = 5000
NUM_FEATURES = 50
NUM_COMPONENTS = 10
RANDOM_STATE = 42

print(f"S3 Bucket: {BUCKET_NAME}")
print(f"S3 Prefix: {PREFIX}")

## Step 2: Generate Synthetic High-Dimensional Data

In [ ]:
# Generate data with many features but only some informative
X, y = make_classification(
    n_samples=NUM_SAMPLES,
    n_features=NUM_FEATURES,
    n_informative=10,       # Only 10 features are informative
    n_redundant=20,          # 20 features are linear combinations
    n_clusters_per_class=3,
    random_state=RANDOM_STATE
)

X = X.astype(np.float32)

print(f"Data shape: {X.shape}")
print(f"Original features: {NUM_FEATURES}")
print(f"Target components: {NUM_COMPONENTS}")
print(f"\nFeature statistics:")
print(f"  Mean range: [{X.mean(axis=0).min():.2f}, {X.mean(axis=0).max():.2f}]")
print(f"  Std range: [{X.std(axis=0).min():.2f}, {X.std(axis=0).max():.2f}]")

In [ ]:
# Visualize correlation matrix
correlation = np.corrcoef(X.T)

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(correlation, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Feature Correlation Matrix')
ax.set_xlabel('Feature')
ax.set_ylabel('Feature')
plt.colorbar(im, ax=ax)
plt.show()

print("Many features are correlated - ideal for PCA!")

## Step 3: Prepare Data for PCA

PCA expects features only in CSV or RecordIO format.

In [ ]:
# Save as CSV
os.makedirs('data/pca', exist_ok=True)

np.savetxt('data/pca/train.csv', X, delimiter=',')

print(f"Saved: data/pca/train.csv ({os.path.getsize('data/pca/train.csv') / 1024:.1f} KB)")

In [ ]:
# Upload to S3
s3_client = boto3.client('s3')

train_s3_key = f"{PREFIX}/train/train.csv"
s3_client.upload_file('data/pca/train.csv', BUCKET_NAME, train_s3_key)

train_uri = f"s3://{BUCKET_NAME}/{PREFIX}/train"
print(f"Data uploaded to: {train_uri}")

## Step 4: Train PCA Model

### Understanding PCA Hyperparameters

| Parameter | Description | Default | Recommendation |
|-----------|-------------|---------|----------------|
| `num_components` | Number of components to compute | Required | Based on variance needs |
| `feature_dim` | Number of input features | Required | Must match data |
| `mini_batch_size` | Batch size | 500 | 200-1000 |
| `algorithm_mode` | `regular` or `randomized` | regular | See below |
| `subtract_mean` | Center data before PCA | True | Almost always True |
| `extra_components` | Extra components (randomized mode) | -1 | 0-10 for stability |

### Mode Selection

| Mode | Complexity | Best For |
|------|------------|----------|
| `regular` | O(min(n²d, nd²)) | Moderate dimensions (<5000 features) |
| `randomized` | O(nd × num_components) | Very large dimensions (>5000 features) |

**Regular mode**: Computes exact PCA using SVD decomposition
**Randomized mode**: Approximate PCA, faster for large matrices

### Key Hyperparameter Details

**num_components**
- How many principal components to keep
- Lower = more compression, potential info loss
- Higher = less compression, more info retained
- Typically choose based on cumulative variance threshold (e.g., 95%)
- For visualization: use 2 or 3

**subtract_mean**
- Centers data by subtracting feature means
- **Almost always True** - standard PCA requires centered data
- Only set False if data is already centered

**mini_batch_size**
- Controls memory usage during training
- Larger = faster, more memory
- Smaller = slower, less memory
- 500 is good default

### Choosing Number of Components

| Method | Description | Threshold |
|--------|-------------|-----------|
| **Variance threshold** | Keep components explaining X% variance | 90-99% |
| **Elbow method** | Plot variance, find bend | Visual |
| **Kaiser criterion** | Keep components with eigenvalue > 1 | For standardized data |
| **Task-specific** | Based on downstream model needs | Varies |

### CloudWatch Training Metrics

| Metric | Description |
|--------|-------------|
| (None reported) | PCA doesn't emit CloudWatch metrics during training |

In [ ]:
# Get PCA container image
pca_image = retrieve(
    framework='pca',
    region=region,
    version='1'
)

print(f"PCA Image URI: {pca_image}")

In [ ]:
# Create PCA estimator
pca_estimator = Estimator(
    image_uri=pca_image,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{BUCKET_NAME}/{PREFIX}/output',
    sagemaker_session=sagemaker_session,
    base_job_name='pca'
)

In [ ]:
# Set hyperparameters
hyperparameters = {
    "num_components": NUM_COMPONENTS,
    "feature_dim": NUM_FEATURES,
    "mini_batch_size": 500,
    "algorithm_mode": "regular",
    "subtract_mean": "True",
}

pca_estimator.set_hyperparameters(**hyperparameters)

print("PCA hyperparameters:")
for k, v in hyperparameters.items():
    print(f"  {k}: {v}")

In [ ]:
# Start training
print("Starting PCA training job...")
print("This will take approximately 3-5 minutes.\n")

pca_estimator.fit(
    {'train': train_uri},
    wait=True,
    logs=True
)

In [ ]:
# Get training job info
job_name = pca_estimator.latest_training_job.name
print(f"Training job completed: {job_name}")
print(f"Model artifacts: {pca_estimator.model_data}")

## Step 5: Deploy and Transform Data

In [ ]:
# Deploy the model
print("Deploying PCA model...")
print("This will take approximately 5-7 minutes.\n")

pca_predictor = pca_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=f'pca-{datetime.now().strftime("%Y%m%d%H%M")}'
)

print(f"\nEndpoint deployed: {pca_predictor.endpoint_name}")

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Configure predictor
pca_predictor.serializer = CSVSerializer()
pca_predictor.deserializer = JSONDeserializer()

def transform_data(data, predictor, batch_size=500):
    """
    Transform data to principal components.
    """
    projections = []
    
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        response = predictor.predict(batch)
        
        for pred in response['predictions']:
            projections.append(pred['projections'])
    
    return np.array(projections)

In [ ]:
# Transform data
print("Transforming data to principal components...")
X_transformed = transform_data(X, pca_predictor)

print(f"\nOriginal shape: {X.shape}")
print(f"Transformed shape: {X_transformed.shape}")
print(f"Dimensionality reduction: {NUM_FEATURES} -> {NUM_COMPONENTS}")

## Step 6: Analyze Principal Components

In [ ]:
# Visualize first 2 principal components
fig, ax = plt.subplots(figsize=(10, 7))

scatter = ax.scatter(X_transformed[:, 0], X_transformed[:, 1], c=y, cmap='tab10', alpha=0.6)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title('Data Projected onto First 2 Principal Components')
plt.colorbar(scatter, ax=ax, label='Class')
plt.show()

In [ ]:
# Variance in each component
component_variance = np.var(X_transformed, axis=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Variance per component
axes[0].bar(range(NUM_COMPONENTS), component_variance)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Variance')
axes[0].set_title('Variance per Principal Component')
axes[0].set_xticks(range(NUM_COMPONENTS))

# Cumulative variance
cumulative_variance = np.cumsum(component_variance) / np.sum(component_variance)
axes[1].plot(range(NUM_COMPONENTS), cumulative_variance, 'bo-')
axes[1].axhline(y=0.9, color='r', linestyle='--', label='90% variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Variance Explained')
axes[1].set_title('Cumulative Variance Explained')
axes[1].set_xticks(range(NUM_COMPONENTS))
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nVariance explained by {NUM_COMPONENTS} components: {cumulative_variance[-1]:.2%}")

In [ ]:
# 3D visualization
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(X_transformed[:, 0], X_transformed[:, 1], X_transformed[:, 2], 
                    c=y, cmap='tab10', alpha=0.6)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('First 3 Principal Components')
plt.colorbar(scatter, ax=ax, label='Class', shrink=0.5)
plt.show()

## Step 7: Clean Up Resources

In [ ]:
# Delete the endpoint
print(f"Deleting endpoint: {pca_predictor.endpoint_name}")
pca_predictor.delete_endpoint()
print("Endpoint deleted successfully!")

---

## Summary

In this exercise, you learned:

1. **Data Format**: CSV with features only (no labels)

2. **Key Hyperparameters**:
   - `num_components`: Target dimensions (most critical)
   - `feature_dim`: Input dimensions
   - `algorithm_mode`: regular or randomized
   - `subtract_mean`: Center data (almost always True)

3. **Output**: Projections onto principal components

4. **Analysis**:
   - Variance per component
   - Cumulative variance explained
   - Visualization in reduced space (2D/3D)

### Instance Recommendations

| Task | Instance Types | Notes |
|------|----------------|-------|
| Training (small) | ml.m5.large | CPU, <5000 features |
| Training (large) | ml.p2.xlarge, ml.p3.2xlarge | GPU, >5000 features |
| Inference | ml.m5.large, ml.c5.large | CPU sufficient |

### Choosing Number of Components

| Target Variance | Typical Use Case |
|-----------------|------------------|
| 70-80% | Fast, lossy compression |
| 90-95% | Good balance (recommended) |
| 99%+ | Near-lossless, noise reduction |
| 2-3 | Visualization only |

### SageMaker PCA Advantages

- GPU acceleration for large matrices
- Single-pass streaming algorithm
- Near-linear scalability
- Randomized mode for huge datasets
- Supports incremental updates

### PCA vs Other Dimensionality Reduction

| Method | Type | Best For |
|--------|------|----------|
| **PCA** | Linear | General purpose, speed |
| t-SNE | Non-linear | Visualization, clusters |
| UMAP | Non-linear | Visualization, preserves global structure |
| Autoencoders | Non-linear | Complex patterns, deep learning |

### Best Practices

1. **Standardize features**: If features have different scales, standardize first!
2. **Check variance explained**: Ensure you retain enough information
3. **Use regular mode** unless features > 5000 or samples are huge
4. **Visualize components**: Plot first 2-3 to understand structure
5. **Watch for outliers**: PCA is sensitive to outliers

### Applications of PCA Output

| Application | How to Use |
|-------------|------------|
| Feature reduction | Use projections as input to ML models |
| Visualization | Plot first 2-3 components |
| Noise reduction | Reconstruct using only top components |
| Anomaly detection | High reconstruction error = anomaly |
| Data compression | Store reduced representation |

### Next Steps

- Use PCA output as input to other ML models
- Combine with K-Means for clustering in reduced space
- Apply for anomaly detection using reconstruction error
- Visualize high-dimensional data in 2D/3D
- Compare with other dimensionality reduction (t-SNE, UMAP)